In [1]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BokehControl
from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div

from datetime import datetime,timedelta
import pandas_datareader as pdr


In [2]:
from bokeh.plotting import figure, output_file, show



# Bokeh Components
In this notebook we will outline how to use one of the most basic Bokeh Components, the button. Other notebooks show more exciting features, and are each designed to be independent. Bokeh Componets is an open source library that allows python developers to create interactive we applications with Bokeh more quickly. It was born out of frustration fighting with analytics libraries, and out of a need to share our analytics process internally. We are happy to share these time saving object oriented libraries with the world.

### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable. Having said that, we have developed a suite of unit tests which we we commit to passing, at least internally.

### The Dashboard
Every Bokeh component has be be embedded into an application. To do this we extend from the basic dashboard application, and add in our button 

In [3]:
class BlankDashboard(Dashboard):
    def getLayout(self):
        l = layout([Div(text="Example <b>HTML</b>", width=200, height=100)],sizing_mode='fixed')         
        return l
d = BlankDashboard()
d.showDashboard()

Loading BokehJS ...

### A Basic Graph
The most basic analytics applications you will create will likely be broken into tabs. In the folliwng example we illustrate how to create a couple of tabs

In [4]:
class GraphDashboard(Dashboard):
    def getLayout(self):
        p = figure(plot_width=400, plot_height=400)
        # add a circle renderer with a size, color, and alpha
        p.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
        l = layout([p],sizing_mode='fixed')         
        return l
d = GraphDashboard()
d.showDashboard()

Loading BokehJS ...

### Defining a custom Graphic
It is possible to create a resuable graphic with event driven & linked data soruces. Right now, this kind of abstraction may not seem interesting, but if this template is followed for all graphics then other features such as linked datasets and update events can be supported instantly. It is reccomended that you follow this template to create a graphic

In [28]:

class TimeseriesGraphic(BokehGraphic):

    def setPlotData(self):    
        
        start_init=datetime(2017,1,26)
        end_init=datetime(2017,4,26)
        
        try: #to increment the day
            self.day_delta = self.day_delta + 1
        except:
            self.day_delta = 0

        # Grab data
        start=start_init + timedelta(days=self.day_delta)
        end=end_init + timedelta(days=self.day_delta)

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]
        ds1_d = dict(x=date_time, y=open_list)
        ds2_d = dict(x=date_time, y=close_list)
        
        self.ds1 = ColumnDataSource(ds1_d)
        self.ds2 = ColumnDataSource(ds2_d)
        
        
        try: #to refresh the data
            self.ds1.data['x'].append(self.date_time[len(self.date_time)-1])
            self.ds1.data['y'].append(self.open_list[len(self.date_time)-1])
            self.ds2.data['x'].append(self.date_time[len(self.date_time)-1])
            self.ds2.data['y'].append(self.close_list[len(self.date_time)-1])
            self.ds1.trigger('data', self.ds1.data, self.ds1.data)
            self.ds2.trigger('data', self.ds2.data, self.ds2.data)  
            
        except Exception as e:
            print(e)
 
    
    
    def createBokehComponent(self):
    
        p=figure(x_axis_type='datetime',plot_width=500,plot_height=500,title="Financial Analysis",tools="",
                      toolbar_location=None)

        r1 = p.line(x='x',y='y',legend="Open Price",color="green", alpha=0.5,source = self.ds1)
        r2 = p.line(x='x',y='y',legend="Open Price",color="red", alpha=0.5,source = self.ds2)
        p.legend.location = "bottom_right"
        return(p)       

In [29]:
class GraphDashboard(Dashboard):
    def getLayout(self):
        self.g = TimeseriesGraphic()
        l = layout([self.g.getBokehComponent()],sizing_mode='fixed')         
        return l    
d = GraphDashboard()
d.showDashboard()

Loading BokehJS ...

'TimeseriesGraphic' object has no attribute 'date_time'


'TimeseriesGraphic' object has no attribute 'date_time'


In [30]:
class TwoGraphDashboard(Dashboard):
    def getLayout(self):
        self.g1 = TimeseriesGraphic()
        self.g2 = TimeseriesGraphic()
        
        l = layout([[self.g1.getBokehComponent(),self.g2.getBokehComponent()]],sizing_mode='fixed')         
        return l    
d = TwoGraphDashboard()
d.showDashboard()

Loading BokehJS ...

'TimeseriesGraphic' object has no attribute 'date_time'
'TimeseriesGraphic' object has no attribute 'date_time'


'TimeseriesGraphic' object has no attribute 'date_time'
'TimeseriesGraphic' object has no attribute 'date_time'
